# Capitulo 4: Spark SQL and DataFrames: Introduction to Built-in Data Sources

Iremos leyendo y realizando los ejemplos del capitulo 4 del libro, complementandolo con unos actividades sobre los mismos ejemplos. 

##### Siempre debemos iniciar una instancia SparkSession al principio. 

Por lo que antes de comenzar crearemos la SparkSession correspondiente:

In [1]:
import pyspark
from pyspark.sql import SparkSession

# Creamos SparkSession (IMPORTANTE cambiar el nombre si estamos en otros scripts)
spark = SparkSession\
    .builder\
    .appName("LibroSpark_cap4")\
    .getOrCreate()


En este capitulo y el siguiente exploraremos como funciona la interfaz SparkSQL con alguno de sus componentes externos. Gracias a SparkSQL:
- Podemos usar las APIS estructuradas de alto y bajo nivel vistas anteriormente (RDD, DF y DS)
- Podemos leer y escribir datos en diferentes formatos (JSON, Tablas Hive, Parquet, Avro, etc)
- Podemos hacer consultas mediante conectores JDBC/OBDC de fuente externa con aplicaciones de Business Inteligence como Tableau, PowerBI, RDBMSs como MySQL o PostgreSQL, etc
- Tenemos una interfaz de programacion para interactuar con datos estructurados almacenados como tablas o vistas en una bbdd de la Spark Aplication
- Podemos interactuar con una shell en SQL para hacer querys, soportando el lenguaje ANSI SQL:2003 y HQL

<center><img src="./images/SparkSQL_stack.PNG"></center>

Recordemos que para usar SparkSQL en la Spark Aplication utilizamos la SparkSession, que hace como punto de entrada para programar con Spark con el uso de APIS. Para usar querys SQL, utilizamos el metodo sql() sobre la instancia de la SparkSession (spark, ver la SparkSession arriba)

## Basic Query Examples

Para esta seccion utilizaremos el dataset departuredelays.csv que contiene datos sobre vuelos de US como fecha, retraso, distancia, origen y destino. Leeremos primero el DF y lo guardaremos como vista temporal para poder ejecutar querys en SQL.

In [2]:
# Cargamos el dataframe departuredelays.csv infiriendo su schema
df = (spark.read.format("csv")
.option("inferSchema", "true")
.option("header", "true")
.load("./Datasets/departuredelays.csv"))

# Comprobamos schema
df.printSchema()

# Si no estuviera bien inferido, se puede crear a través de lenguaje DDL y cambiando el inferSchema por este schema
schema = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"

# Creamos vista temporal de DF cargado con createOrReplaceTempView
df.createOrReplaceTempView("us_delay_flights_tbl")

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



Ya tenemos nuestra vista temporal a través del DF cargado de un archivo csv. Podemos usar sentencias en lenguaje SQL tal y como haríamos con MySQL o PostgreeSQL con el método spark.sql(sentencia). Algunos ejemplos:

In [3]:
from pyspark.sql.types import *

# Utilizamos el metodo el método spark.sql(sentencia)
# Importante utilizar las triples comillas """""" para poder ordenar en varias lineas nuestra consulta
# Ejemplos de algunas consultas sencillas

# 1
spark.sql("""SELECT distance, origin, destination
             FROM us_delay_flights_tbl 
             WHERE distance > 1000
             ORDER BY distance DESC""").show(10)
# 2
spark.sql("""SELECT date, delay, origin, destination
             FROM us_delay_flights_tbl
             WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
             ORDER by delay DESC""").show(10)

# Ejercicio adicional:
# convierta la columna de fechas en un formato legible (con CAST)
spark.sql("""SELECT CAST(date as timestamp) as Date
             FROM us_delay_flights_tbl """).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows

+-------------------+
|               

In [5]:
# 3 Mostramos un ejemplo de una consulta más completa utilizando la clausula CASE
spark.sql("""SELECT delay, origin, destination,
             CASE
                 WHEN delay > 360 THEN 'Very Long Delays'
                 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
                 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
                 WHEN delay = 0 THEN 'No Delays'
                 ELSE 'Early'
             END AS Flight_Delays
             FROM us_delay_flights_tbl
             ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



Como hemos comentado varias veces, estas consultas de SQLpuro escritas con la interfaz SparkSQL pueden ser escritas de manera equivalente en la API de Dataframe con identico resultado y rendimiento gracias al CatalystOptimizer:

In [4]:
from pyspark.sql.functions import *

# 1
(df.select("distance", "origin", "destination")
   .where("distance > 1000")
   .orderBy("distance", ascending=False)
   .show(10))

# 2
(df.select("date", "distance", "origin", "destination")
   .where( (df.delay > 120) & (df.origin == 'SFO') & (df.destination == 'ORD') )
   .orderBy("distance", ascending=False)
   .show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows

+-------+--------+------+-----------+
|   date|distance|origin|destination|
+-------+--------+------+-----------+
|3201100|    1604|   SFO|        ORD|
|3311810|    1604|   SFO|        ORD|
|3311405|    1604|   SFO|        ORD|
|3120929|    1604|   SFO|        ORD|
|3141657|    1604|   SFO|        ORD|
|3171251|    1604|   SFO|        ORD|
|3171215|    1604|   SFO|        ORD|
|3260828|    1604|   SFO|        ORD|
|3261106|    1604|   SFO|        ORD|
|3272225|    1604|   SFO|        ORD|
+-------+--------+------+-----------+
only showing top 10 ro

## SQL Tables and Views

En esta parte veremos como se crean las tablas y vistas y como las gestiona Spark, tanto en memoria como en disco. Las tablas contienen datos y asociada a cada tabla estan los metadatos, que es la información que describen el contenido de los datos de la tabla como el shcema, descripción, nombre de la tabla, particiones, localización, etc. Los metadatos se almacenan en un metastore central.

En lugar de tener un metastore separado para las tablas de Spark, Spark utiliza por defecto el metastore de Apache Hive, ubicado en **/user/hive/warehouse**, para guardar todos los metadatos sobre sus tablas. Puedes uedes cambiar la ubicación por defecto estableciendo la variable de configuración de Spark **spark.sql.warehouse.dir** a otra ubicación, pdudiendose estableceser en local o de manera externa.


### Managed vs Unmanaged Tables Differences

Puedes crea dos tipos de tablas: **Managed o dirigidas y Unmanaged o no dirigidas.**

Para las tablas dirigidas, Spark maneja los metadatos y los datos en el file store, que puede ser en Local, HDFS o otros stores como AmazonS3 o Azure Blob. Para el caso de las no dirigidas Spark solo gestiona los metadatos y tu los gestionas en una fuente de datos externa como Cassandra.

En una tabla dirigida, sentencias SQL como DROP TABLE borran los datos y los metadatos mientras que en una tabla no dirigida el mismo comando elimina solo los metadatos y no los datos. 


## Creating SQL Databases and Tables

Por defecto, spark crea las tablas en la bbdd default. Para crear una bbdd podemos usar el lenguaje SQL, en el siguiente ejemplo creamos la bbdd learn_spark_db y la marcamos para que la siguientes tablas que creemos se almacenen en dicha bbdd.



In [5]:
# Creamos bbdd learn_spark_db y la marcamos
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

### Creating a managed table


In [6]:
# Creamos la tabla con SQL

spark.sql("""CREATE TABLE IF NOT EXISTS managed_us_delay_flights_tbl (
          date STRING, delay INT,distance INT, origin STRING, destination STRING )""")

# Lanza una excepción que indica que se cree la tabla como CTA

AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);;
'CreateTable `managed_us_delay_flights_tbl`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore


In [7]:
spark.sql( """CREATE TABLE IF NOT EXISTS managed_us_delay_flights_tbl
              AS SELECT * FROM us_delay_flights_tbl""")

# Lanza otra excepción

AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);;
'CreateTable `managed_us_delay_flights_tbl`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore
+- Project [date#16, delay#17, distance#18, origin#19, destination#20]
   +- SubqueryAlias us_delay_flights_tbl
      +- Relation[date#16,delay#17,distance#18,origin#19,destination#20] csv


In [13]:
# Creamos la tabla con DF api, cargando primero el DF flights_df del df original y guardandolo con write.saveAsTable
flights_df = df
flights_df.write.saveAsTable("managed_us_delay_flights_tbl_apidf")


### Creating a unmanaged table 

Puedes crear tablas no dirigidas en diferentes formatos como csv, parquet o json accesibles en el file store local de la Spark Aplication. 

En el siguiente ejemplo creamos una tabla unmanaged de un archivo csv, se pueden ver las dos maneras, SQLpuro y API DF

In [8]:
# SQL puro
spark.sql(""" CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, distance INT, origin STRING, destination STRING)
              USING csv OPTIONS (PATH'./Datasets/departuredelays.csv') """)

DataFrame[]

In [11]:
# DF api (modificamos nombre a us_delay_flights_tbl2 para que no lance error)
(df.write
   .option("path", "./Datasets/departuredelays.csv")
   .saveAsTable("us_delay_flights_tbl2"))

## Creating Views

Además, en spark se pueden crear vistas de tablas ya existentes. Las vistas pueden ser:
- Globales (visibles en todas las SparkSessions de un determinado cluster)
- Temporales (desaparecen cuando después de que la aplicación Spark termine)

La sintaxis para la creación de vistas es similar a la de la creación de tablas. La diferencia es entre una vista y una tabla es que la en la **vista los datos de la tabla o se guaran despues de que la spark aplication termine.**

La **diferencia entre las vistas temporales y globales temporales** es que una vista temporal está ligada a una única SparkSession dentro de una aplicación Spark. En cambio, una vista temporal global es visible en múltiples SparkSessions dentro de una aplicación Spark. 

Puede crear múltiples SparkSessions dentro de una sola aplicación Spark. Esto puede ser útil en los casos en los que quieras acceder (y combinar) datos de dos diferentes SparkSessions que no comparten las mismas configuraciones de metastore de Hive.

Puedes crrear vistas de una tabla SQL para por ejemplo trabajar con submuestras de una tabla y no preocuparte en crear, guardar la tabla. Ejemplos:

In [14]:
# En lenguaje SQL puro (no hace falta ejecutae)
spark.sql( """CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view 
              AS SELECT date, delay, origin, destination from us_delay_flights_tbl 
              WHERE origin = 'SFO' """)

spark.sql( """CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view 
              AS SELECT date, delay, origin, destination from us_delay_flights_tbl 
              WHERE origin = 'JFK'""" )

In [15]:
# En API dF
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
                   
# Creamos vista temporal global (createOrReplaceGlobalTempView) y vista temporal (createOrReplaceTempView)
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

Una vez creadas las vistas, puedes hacer querys sobre ellas como si de una tabla se tratase, por lo que es mas comodo si no quieres que se guarden los datos despues de la tabla despues de que la spark aplication termine.

In [17]:
# En SQL (meter entre spark.sql para lanzarlo)
spark.sql( """SELECT * FROM us_origin_airport_JFK_tmp_view""" ).show(5)

# Python
spark.read.table("us_origin_airport_JFK_tmp_view").show(5)


+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows



Tambien puedes borrar la vista, ejemplo:

In [18]:
# En SQL (meter entre spark.sql para lanzarlo)
 # DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view;
 # DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view

# API DF
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

## Viewing the Metadata

Para manejar los metadatos de cada tabla (managed o unmanaged) utilizamos el Catalog, una abtración de SparkSQL para el almacenamiento de los metadatos. 

Por ejemplo, en una Spark Aplication, despues de crear la spark = SparkSession puedes acceder al almacen de metadatos con estos metodos:

In [23]:
# metodo listdatabases de catalog
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/C:/Users/franciscomanuel.medi/OneDrive%20-%20Bosonit/Escritorio/Practicas/Practica%20Spark/Notebooks/Libro%20Learning%20Spark%20L20/spark-warehouse'),
 Database(name='learn_spark_db', description='', locationUri='file:/C:/Users/franciscomanuel.medi/OneDrive%20-%20Bosonit/Escritorio/Practicas/Practica%20Spark/Notebooks/Libro%20Learning%20Spark%20L20/spark-warehouse/learn_spark_db.db')]

In [24]:
# metodo listTables de catalog
spark.catalog.listTables()

[Table(name='us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [27]:
# metodo listTables de catalog
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

## Caching SQL Tables

Puedes cachear y descachear las tablas de SQL y vistas. Puedes especificar una tabla como LAZY, siginifcando esto que se sólo debe almacenarse en la caché cuando se utiliza por primera vez en lugar de inmediatamente:

In [ ]:
# En SQL (meter entre spark.sql para lanzarlo)
 # CACHE [LAZY] TABLE <table-name>
 # UNCACHE TABLE <table-name>

## Reading Tables into DataFrames

Hemos visto anteriormente que SparkSQL tiene una amplia variedad a la hora de guardar o leer tanto tablas como DF en diferentes formatos. A su vez, Spark distingue entre:
- DataFrameReader para leer datos en diferentes formatos
- DataFrameWritter para guardar datos en diferentes formatos

### DataFrameReader
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader

Podemos leer datos, ya sean DF, archivos, o tablas en base a diferentes formatos, para ver los argumentos del DataFrameReader en el link adjunto de la documentación de spark.

Hemos utilizado el DFreader cada vez que hemos creado un DF a través de un archivo, ya sea CSV, Parquet, Json, etc en anteriores notebooks

### DataFrameWritter
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter

Podemos guardar datos, ya sean DF, archivos, o tablas en base a diferentes formatos, para ver los argumentos del DataFrameReader en el link adjunto de la documentación de spark.

Hemos utilizado el DFwriter cada vez que hemos creado un DF a través de un archivo, ya sea CSV, Parquet, Json, etc en anteriores notebooks

Se pueden ver **más ejemplos de lectura en Ejs_word_cap4 y guardado de DFs en Ejs_word_cap3**

#### Formato Parquet
https://sparkbyexamples.com/pyspark/pyspark-read-and-write-parquet-file/

El formato columnar parquet es el formato por defecto en Spark. Es la mas soportada por diferentes plataformas y frameworks, open source y la hacen la más optimizada y eficiente, sobre todo cuando el DF contiene muchas columnas. Es recomendable guardar los DFs limpios y transformados en este formato.

Parquet es guardado en un directorio que contiene la estructura de los datos, los metadatos y un numero de archivos (dependiendo del numero de particiones que tenga spark para procesar).

**Es muy importante destacar que para este tipo de archivos parquet NO es necesario especificar un schema por que está incluido en los metadatos**

In [ ]:
# Read mnm_df en formato parquet
df_parquet = spark.read.format("parquet").load("./Datasets/DFs_saved/mnm_df_parquet/*").show(5)

In [58]:
# Guardar DF como parquet
( mnm_df.write.mode('overwrite') 
            .parquet("./Datasets/DFs_saved/mnm_df_parquet") )
# *Como parquet es el formato por defecto, si no incluyes nada en .format(), este lo creará por defect como parquet

In [33]:
# Creamos vista temporal us_delay_flights_tbl del archivo /flightsDF_parquet
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING parquet
OPTIONS (path "./Datasets/DFs_saved/flightsDF_parquet")""")

# Podemos hacer sentencias sql sobre esta tabla us_delay_flights_tbl
spark.sql("SELECT * FROM us_delay_flights_tbl")#.show(5)

DataFrame[Month: int, DayofMonth: int, DayOfWeek: int, FlightDate: string, Origin: string, OriginCity: string, Dest: string, DestCity: string, DepTime: int, DepDelay: double, ArrTime: int, ArrDelay: double, Cancelled: double, CancellationCode: string, Diverted: double, ActualElapsedTime: double, AirTime: double, Distance: double, CarrierDelay: double, WeatherDelay: double, NASDelay: double, SecurityDelay: double, LateAircraftDelay: double]

In [ ]:
# Guardar tabla us_delay_flights_tbl en formato parquet
(df.write
.mode("overwrite")
.saveAsTable("us_delay_flights_tbl"))

#### Formatos: Json

JavaScript Object Notation (JSON) tambien es un formato popular y dacil de leer y parsear comparado con XML. Consta de 2 representaciones: Single-Line Mode y Multiline Mode.

En Single-Line Mode cada linea es un objeto JSON, mientras que en Multi-Line Mode el archivo entero es un objeto JSON. Hay que marcar True en option() para usar este ultimo metodo.

Ejemplos para leer y guardar un archivo JSON como DF

In [62]:
# leer archivo blogs.json
df_json = spark.read.format("json").load("./Datasets/blogs.json")

In [ ]:
# write/save df en formato json
(df_json.write.format("json")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("./Datasets/DFs_saved/mnm_df_json"))

Ejemplos para leer o guardar archivos JSON como una tabla o vista SQL

In [36]:
# Creamos vista temporal us_delay_flights_tbl del archivo blogs.json
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING json
OPTIONS (path "./Datasets/blogs.json")""")

# Podemos hacer sentencias sql sobre esta tabla us_delay_flights_tbl
spark.sql("SELECT * FROM us_delay_flights_tbl")#.show(5)

DataFrame[Campaigns: array<string>, First: string, Hits: bigint, Id: bigint, Last: string, Published: string, Url: string]

#### Formatos: CSV

Los archivos de texto plano son muy comunes y popular, suelen tener la coma como separador por defecto.

Ejemplos para leer y guardar un archivo CSV como DF

In [40]:
# Cargamos el dataframe departuredelays.csv infiriendo su schema
df = (spark.read.format("csv")
.option("inferSchema", "true")
.option("header", "true")
.load("./Datasets/departuredelays.csv"))

# write/save df en formato CSV
df.write.format("csv").mode("overwrite").save("./Datasets/df_csv")

Ejemplos para leer o guardar archivos CSV como una tabla o vista SQL

In [41]:
# Creamos vista temporal us_delay_flights_tbl del archivo departuredelays.csv
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING csv
OPTIONS (
path "./Datasets/departuredelays.csv*",
header "true",
inferSchema "true",
mode "FAILFAST" )""")

# Podemos hacer sentencias sql sobre esta tabla us_delay_flights_tbl
spark.sql("SELECT * FROM us_delay_flights_tbl")#.show(5)

DataFrame[]

#### Formatos: Avro

Avro es un formato por Apache Kafka para serializzar y deserializar. Tiene algunas ventajas como el mapeo directo a JSON, velocidad y eficiencia. Tambien es utilizado por muchos lenguajes de programacion.

Ejemplos para leer y guardar un archivo avro como DF

In [ ]:
# leer archivo /flightsDF_avro
df = spark.read.format("parquet").load("./Datasets/DFs_saved/flightsDF_avro")

# write/save df en formato avro
(df.write.format("avro")
   .mode("overwrite")
   .option("compression", "snappy")
   .save("./Datasets/DFs_saved/flightsDF_avro"))


#### Formatos: ORC

Es otro formato de archivo columnar optimizado adicional. 

Ejemplos para leer y guardar un archivo ORC como DF

In [46]:
# leer archivo 2010-summary.orc
file = "./Datasets/2010-summary.orc/*"
df = spark.read.format("orc").option("path", file).load()

# write/save df en formato orc
(df.write.format("orc")
.mode("overwrite")
.option("compression", "snappy")
.save("./Datasets/DFs_saved/df_orc"))

#### Formatos: Images

Desde Spark 2.4 se introdujo como formato los archivos de imagen para dar soporte  alos entornos de machine learning y deep learning como Tensorflow o Pytorch. Es importante para estos ambitos cargar y procesar datasets de imagen en este formato

Antes de cargar el archivo de tipo image hay que importar este tipo de archivos desde el paquete ml. Ejemplo:

In [48]:
# Leer archivo train_images en formato image
from pyspark.ml import image

image_dir = "./Datasets/cctvVideosasets/learning-spark-v2/cctvVideos/train_images"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()


In [12]:
spark.stop()